<a href="https://colab.research.google.com/github/Djadih/ECE381V-Genomic-SP-DS-HW5/blob/main/hw5_p1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import numpy as np
np.set_printoptions(linewidth=np.inf)

In [19]:
f = open("SNP_Fragment_Matrix.txt",'r')

hap_1 = {}
hap_2 = {}
R = np.zeros((80,194))
for line_num,line in enumerate(f):
  for idx, chr in enumerate(line.strip()):
    if chr != "-":
      if idx not in hap_1:
        hap_1[idx] = chr
        R[line_num,idx] = 1
      if idx in hap_1:
        if hap_1[idx] == chr:
          R[line_num,idx] = 1
        else:
          R[line_num,idx] = -1
          hap_2[idx] = chr

In [46]:
len(hap_2)

194

In [21]:
mask = R!=0

In [22]:
R

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [-1.,  1., -1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [23]:
mask = mask.astype(np.int32)
mask

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [24]:
(mask * R) == R

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [25]:
np.set_printoptions(linewidth=np.inf)
for idx, vec in enumerate(R):
  print(idx, np.count_nonzero(vec==-1))


0 0
1 3
2 4
3 4
4 5
5 8
6 4
7 8
8 5
9 6
10 5
11 7
12 7
13 5
14 8
15 9
16 9
17 4
18 7
19 6
20 6
21 4
22 10
23 8
24 6
25 4
26 8
27 7
28 5
29 6
30 7
31 8
32 5
33 7
34 8
35 7
36 6
37 7
38 8
39 7
40 20
41 19
42 20
43 19
44 18
45 19
46 16
47 23
48 19
49 18
50 20
51 16
52 23
53 17
54 18
55 18
56 21
57 18
58 21
59 15
60 20
61 18
62 17
63 18
64 21
65 17
66 19
67 22
68 16
69 18
70 15
71 21
72 15
73 18
74 19
75 19
76 15
77 20
78 17
79 17


In [26]:
def f(mask,R,U,V):
  return np.linalg.norm(mask * (R - U @ V.T))

def argmin_F_U(mask, R, U, V):
  for row in range(U.shape[0]):
    U_copy = U.copy()
    U_copy[row] = U_copy[row] ^ 1
    f_orig = f(mask,R,U,V)
    f_mod = f(mask,R,U_copy,V)
    if f_mod < f_orig:
      U = U_copy
  return U


In [27]:
alpha = 0.02
n = 80
m = 194
k = 2
#init U and V
U = np.random.randint(0,2,size = (n,1))
U = np.concatenate([U,np.bitwise_xor(U,1)],axis=1)

V = (np.random.randint(0,2,size = (m,1))*2-1).astype(np.int32)
V = np.concatenate([V,-V],axis=1)

for iteration in range(500):
  d_f = -2 * (mask * (R - U @ V.T)).T @ U
  V = V - alpha * d_f
  U = argmin_F_U(mask, R, U, V)


In [36]:
U
print(U[39])
print(U[40])

[1 0]
[0 1]


In [29]:
V

array([[-5.51603196e-17, -1.00000000e+00],
       [ 1.00000000e+00,  6.66666667e-01],
       [-7.14285714e-01, -1.00000000e+00],
       [ 1.00000000e+00,  7.14285714e-01],
       [ 1.00000000e+00,  7.50000000e-01],
       [ 8.00000000e-01,  1.00000000e+00],
       [ 8.00000000e-01,  1.00000000e+00],
       [ 8.18181818e-01,  1.00000000e+00],
       [ 1.00000000e+00, -1.00000000e+00],
       [ 1.00000000e+00,  7.77777778e-01],
       [ 1.00000000e+00,  7.77777778e-01],
       [ 1.00000000e+00,  8.66666667e-01],
       [ 1.00000000e+00, -1.00000000e+00],
       [ 1.00000000e+00, -1.00000000e+00],
       [ 1.00000000e+00,  8.88888889e-01],
       [ 1.00000000e+00,  8.88888889e-01],
       [ 9.04761905e-01,  1.00000000e+00],
       [ 1.00000000e+00, -1.00000000e+00],
       [ 8.26086957e-01,  1.00000000e+00],
       [ 1.00000000e+00,  9.04761905e-01],
       [ 9.13043478e-01,  1.00000000e+00],
       [ 9.13043478e-01,  1.00000000e+00],
       [ 9.13043478e-01,  1.00000000e+00],
       [ 1.

In [30]:
V_quant = np.rint(V)

In [31]:
V_quant[0,1] = 1
V_quant

array([[-0.,  1.],
       [ 1.,  1.],
       [-1., -1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1., -1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1., -1.],
       [ 1., -1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1., -1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1., -1.],
       [ 1.,  1.],
       [-1., -1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1., -1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,

In [32]:
h1 = V_quant.T[0]
h2 = V_quant.T[1]

In [33]:
s = ''
for i, j in enumerate(h2):
  if(j==1):
    s+=hap_1[i]
  if(j==-1):
    s+=hap_2[i]

In [41]:
s = ''
for i in range(194):
  s += str(hap_1[i])

In [42]:
s

'AGCCTATCGGTATAAAGCCTCGTGAGAGCCATCCGGACATCCGATGAGAAGGAGTTCAGCTTACCCTACACGATCGGCTCTTGCCACGGCTGGAGGTACTGTCGCGCCCCCAAATTACGATTGTCCTTGGAGACAGCCGGTTTCGCAGCTGCGAAAACGATCCGGTCGATTTCCGTAGGAAGGGAAAGCCCGTA'

In [43]:
def hd(row, h):
  d = 0
  for idx,i in enumerate(row):
    if(i != 0):
      if(i!=h[idx]):
        d+=1
  return d

def MEC(R,h1,h2):
  d = 0
  for row in R:
    d+=min(hd(row,h1),hd(row,h2))
  return d

In [44]:
MEC(R,h1,h2)

201